# Keras Tuner Bars

In [ ]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
import time
import sys
import copy

sys.path.append("..")
from helpers import filename_for

In [ ]:
import tensorflow
from tensorflow import keras
import kerastuner

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)
print("keras-tuner", kerastuner.__version__)

In [ ]:
# Sinmply loading everything into memory is requires the least effort, but quite alot of RAM
files = [filename_for(15, 30, 600, 500, n, "inclxx", s, "bars.parquet") for n in [1, 2, 3, 4] for s in range(7)] # should be range(20), not enough ram
data = pd.concat([pd.read_parquet(file) for file in files], ignore_index=True).sample(frac=1)
data.loc[data["nHits"] == 0, "nPN"] = 0
display(data)

In [ ]:
# cols_tri = ["nHits", "nClus", "Edep"]
cols_e = [str(i) for i in range(0, 30 * 100 * 2, 2)]
cols_t = [str(i + 1) for i in range(0, 30 * 100 * 2, 2)]

scaler = sklearn.preprocessing.MaxAbsScaler()
# s_tri = copy.copy(scaler).fit(data[cols_tri])
s_e = copy.copy(scaler).fit(data[cols_e].values.reshape(-1, 1))
s_t = copy.copy(scaler).fit(data[cols_t].values.reshape(-1, 1))

data_scaled = np.concatenate(
    (
        # s_tri.transform(data[cols_tri]),
        s_e.transform(data[cols_e].values.reshape(-1, 1)).reshape(-1, len(cols_e)),
        s_t.transform(data[cols_t].values.reshape(-1, 1)).reshape(-1, len(cols_t)),
    ),
    axis=1,
)
display(data_scaled)

In [ ]:
label = "nPN"
max_neutrons = 4
num_classes = max_neutrons + 1

ys = keras.utils.to_categorical(data[[label]].values.ravel())
Xs = data_scaled

In [ ]:
class Model(kerastuner.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.models.Sequential()
        model.add(keras.layers.Dense(units=hp.Int(f"relu1nodes", min_value=1000, max_value=10000, step=3000), activation="relu",))
        model.add(keras.layers.Dense(units=hp.Int(f"relu2nodes", min_value=500, max_value=3000, step=200), activation="relu",))
        model.add(keras.layers.Dense(units=self.num_classes, activation="softmax"))

        optimizer = hp.Choice("optimizer", ["adam", "adagrad", "adadelta", "SGD"])
        loss = keras.losses.CategoricalCrossentropy()

        model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])

        return model

In [ ]:
model = Model(num_classes)

tuner = kerastuner.tuners.BayesianOptimization(
#tuner = kerastuner.tuners.RandomSearch(
    model, objective="val_accuracy", max_trials=20, executions_per_trial=1, directory="kerastuner", project_name="bars-V1" #, overwrite=True
)
tuner.search_space_summary()

In [ ]:
tuner.search(
    Xs, ys, verbose=2, validation_split=0.1
)

In [ ]:
tuner.results_summary()